# Model Selection

When you have many predictor variables in a predictive model, the model selection methods allow to select automatically the best combination of predictor variables for building an optimal predictive model.

Removing irrelevant variables leads a more interpretable and a simpler model. With the same performance, a simpler model should be always used in preference to a more complex model.

Additionally, the use of model selection approaches is critical in some situations, where you have a large multivariate data sets with many predictor variables. This is often the case in genomic area, where a substantial challenge comes from the fact that the number of genomic variables (p) is usually much larger than the number of individuals (n) (i.e., p >> n) (Bovelstad et al. 2007).

It’s well known that, when p >> n, it is easy to find predictors that perform excellently on the fitted data, but fail in external validation, leading to poor prediction rules. Furthermore, there can be a lot of variability in the least squares fit, resulting in overfitting and consequently poor predictions on future observations not used in model training (James et al. 2014).

One possible strategy consists of testing all possible combination of the predictors, and then selecting the best model. This method called best subsets regression  is computationally expensive and becomes unfeasible for a large data set with many variables.

A better alternative to the best subsets regression is to use the stepwise regression (Chapter @ref(stepwise-regression)) method, which consists of adding and deleting predictors in order to find the best performing model with a reduced set of variables .

Other methods for high-dimensional data, containing multiple predictor variables, include the penalized regression (ridge and lasso regression, and the principal components-based regression methods .

The stepwise regression (or stepwise selection) consists of iteratively adding and removing predictors, in the predictive model, in order to find the subset of variables in the data set resulting in the best performing model, that is a model that lowers prediction error.

There are three strategies of stepwise regression (James et al. 2014,P. Bruce and Bruce (2017)):

1. Forward selection, which starts with no predictors in the model, iteratively adds the most contributive predictors, and stops when the improvement is no longer statistically significant.
2. Backward selection (or backward elimination), which starts with all predictors in the model (full model), iteratively removes the least contributive predictors, and stops when you have a model where all predictors are statistically significant.
3. Stepwise selection (or sequential replacement), which is a combination of forward and backward selections. You start with no predictors, then sequentially add the most contributive predictors (like forward selection). After adding each new variable, remove any variables that no longer provide an improvement in the model fit (like backward selection).

# Penalized Regression Essentials: Ridge, Lasso & Elastic Net

The standard linear model (or the ordinary least squares method) performs poorly in a situation, where you have a large multivariate data set containing a number of variables superior to the number of samples.

A better alternative is the penalized regression allowing to create a linear regression model that is penalized, for having too many variables in the model, by adding a constraint in the equation (James et al. 2014,P. Bruce and Bruce (2017)). This is also known as shrinkage or regularization methods.

The consequence of imposing this penalty, is to reduce (i.e. shrink) the coefficient values towards zero. This allows the less contributive variables to have a coefficient close to zero or equal zero.

Note that, the shrinkage requires the selection of a tuning parameter (lambda) that determines the amount of shrinkage.

# Why shrink or subset and what does this mean?
In the linear regression context, subsetting means choosing a subset from available variables to include in the model, thus reducing its dimensionality. Shrinkage, on the other hand, means reducing the size of the coefficient estimates (shrinking them towards zero). Note that if a coefficient gets shrunk to exactly zero, the corresponding variable drops out of the model. Consequently, such a case can also be seen as a kind of subsetting.

Shrinkage and selection aim at improving upon the simple linear regression. There are two main reasons why it could need an improvement:

* Prediction accuracy: Linear regression estimates tend to have low bias and high variance. Reducing model complexity (the number of parameters that need to be estimated) results in reducing the variance at the cost of introducing more bias. If we could find the sweet spot where the total error, so the error resulting from bias plus the one from variance, is minizmized, we can improve the model’s predictions.

* Model’s interpretability: With too many predictors it is hard for a human to grasp all the relations between the variables. In some cases we would be willing to determine a small subset of variables with the strongest impact, thus sacrificing some details in order to get the big picture.

# Setup & Data Load

In [4]:
# Import necessary modules and set options
import pandas as pd
import numpy as np
import itertools

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

# Load data
data = pd.read_csv("prostate.data", sep = "\t")
print(data.head())

# Train-test split
y_train = np.array(data[data.train == "T"]['lpsa'])
y_test = np.array(data[data.train == "F"]['lpsa'])
X_train = np.array(data[data.train == "T"].drop(['lpsa', 'train'], axis=1))
X_test = np.array(data[data.train == "F"].drop(['lpsa', 'train'], axis=1))

   Unnamed: 0    lcavol   lweight  age  ...  gleason  pgg45      lpsa  train
0           1 -0.579818  2.769459   50  ...        6      0 -0.430783      T
1           2 -0.994252  3.319626   58  ...        6      0 -0.162519      T
2           3 -0.510826  2.691243   74  ...        7     20 -0.162519      T
3           4 -1.203973  3.282789   58  ...        6      0 -0.162519      T
4           5  0.751416  3.432373   62  ...        6      0  0.371564      T

[5 rows x 11 columns]


# Linear Regression

Let us start with the simple linear regression, which will constitute our benchmark. It models the target variable, y, as a linear combination of p predictors, or features, X:

In [6]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/max/564/1*1T261QLMnn9ApzSJZ_6L-w.png")

This model has p + 2 parameters that have to be estimated from the training data:

* The p feature β-coefficients, one per variable, denoting their impacts on the target;
* One intercept parameter, denoted as β0 above, which is the prediction in case all Xs are zero. It is not necessary to include it in the model, and indeed in some cases it should be dropped (e.g. if one wants to include a full set of dummies denoting levels of a categorical variable) but in general it gives the model more flexibility, as you will see in the next paragraph;
* One variance parameter of the Gaussian error term.

These parameters are typically estimated using the Ordinary Least Square (OLS). OLS minimizes the sum of squared residuals, given by

In [7]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/max/964/1*-FDUVzZCw-5OvMOH2Je3hQ.png")

It is helpful to think about this minimization criterion graphically. With only one predictor X, we are in a 2D space, formed by this predictor and the target. In this setting, the model fits such a line in the X-Y space that is the closest to all data points, with the proximity measured as the sum of squared vertical distances of all data points — see the left panel below. If there are two predictors, X1 and X2, the space grows to 3D and now the model fits a plane that is closest to all points in the 3D space — see the right panel below. With more than two features, the plane becomes the somewhat abstract hyperplane, but the idea is still the same. These visualisations also help to see how the intercept gives the model more flexibility: if it is included, it allows the line or plane to not cross the space’s origin.


In [10]:
Image(url= "https://miro.medium.com/max/1400/1*CSrJ-XXGy9iVMaCRDOPFQg.png")

The minimization problem described above turns out to have an analytical solution, and the β-parameters can be calculated as

In [11]:
Image(url= "https://miro.medium.com/max/191/1*SOihShh-7bIvAlsjzL4nUQ.png")

Including a column of ones in the X matrix allows to express the intercept part of the β-hat vector in the formula above. The “hat” above the “β” denotes that it is an estimated value, based on the training data.

# The Bias-Variance trade-off

In statistics, there are two critical characteristics of estimators to be considered: the bias and the variance. The bias is the difference between the true population parameter and the expected estimator. It measures the inaccuracy of the estimates. The variance, on the other hand, measures the spread between them.

In [12]:
Image(url= "https://miro.medium.com/max/896/1*jngqFdNagZhMMoSS5UyHoQ.jpeg")

Clearly, both bias and variance can harm the model’s predictive performance if they are too large. The linear regression, however, tends to suffer from variance, while having a low bias. This is especially the case if there are many predictive features in the model or if they are highly correlated with each other. This is where subsetting and regularization come to rescue. They allow to reduce the variance at the cost of introducing some bias, ultimately reducing the total error of the model.

Before discussing these methods in detail, let us fit linear regression to out prostate data and check its out-of-sample Mean Prediction Error (MAE).

In [5]:
linreg_model = LinearRegression(normalize=True).fit(X_train, y_train)
linreg_prediction = linreg_model.predict(X_test)
linreg_mae = np.mean(np.abs(y_test - linreg_prediction))
linreg_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((linreg_model.intercept_, linreg_model.coef_), 
        axis=None), 3))
)

print('Linear Regression MAE: {}'.format(np.round(linreg_mae, 3)))
print('Linear Regression coefficients:')
linreg_coefs

Linear Regression MAE: 0.24
Linear Regression coefficients:


{'Intercept': 1.014,
 'Unnamed: 0': 0.034,
 'age': -0.007,
 'gleason': -0.042,
 'lbph': 0.046,
 'lcavol': 0.145,
 'lcp': -0.052,
 'lweight': 0.056,
 'pgg45': 0.003,
 'svi': 0.066}

# Best Subset Regression

A straightforward approach to choosing a subset of variables for linear regression is to try all possible combinations and pick one that minimizes some criterion. This is what Best Subset Regression aims for. For every k ∈ {1, 2, …, p}, where p is the total number of available features, it picks the subset of size k that gives the smallest residual sum of squares. However, sum of squares cannot be used as a criterion to determine k itself, as it is necessarily decreasing with k: the more variables are included in the model, the smaller are its residuals. This does not guarantee better predictive performance though. That’s why another criterion should be used to select the final model. For models focused on prediction, a (possibly cross-validated) error on test data is a common choice.

As Best Subset Regression is not implemented in any Python package, we have to loop over k and all subsets of size k manually. The following chunk of code does the job.

In [13]:
results = pd.DataFrame(columns=['num_features', 'features', 'MAE'])

# Loop over all possible numbers of features to be included
for k in range(1, X_train.shape[1] + 1):
    # Loop over all possible subsets of size k
    for subset in itertools.combinations(range(X_train.shape[1]), k):
        subset = list(subset)
        linreg_model = LinearRegression(normalize=True).fit(X_train[:, subset], y_train)
        linreg_prediction = linreg_model.predict(X_test[:, subset])
        linreg_mae = np.mean(np.abs(y_test - linreg_prediction))
        results = results.append(pd.DataFrame([{'num_features': k,
                                                'features': subset,
                                                'MAE': linreg_mae}]))

# Inspect best combinations
results = results.sort_values('MAE').reset_index()
print(results.head())

# Fit best model
best_subset_model = LinearRegression(normalize=True).fit(X_train[:, results['features'][0]], y_train)
best_subset_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((best_subset_model.intercept_, best_subset_model.coef_), axis=None), 3))
)

print('Best Subset Regression MAE: {}'.format(np.round(results['MAE'][0], 3)))
print('Best Subset Regression coefficients:')
best_subset_coefs

   index num_features            features       MAE
0      0            4        [0, 2, 3, 4]  0.221212
1      0            3           [0, 3, 4]  0.222467
2      0            3           [0, 2, 3]  0.222898
3      0            6  [0, 1, 2, 3, 4, 6]  0.224248
4      0            5     [0, 1, 3, 4, 6]  0.224347
Best Subset Regression MAE: 0.221
Best Subset Regression coefficients:


{'Intercept': 0.607,
 'Unnamed: 0': 0.039,
 'age': 0.027,
 'lcavol': 0.025,
 'lweight': -0.002}

# Ridge Regression

One drawback of Best Subset Regression is that it does not tell us anything about the impact of the variables that are excluded from the model on the response variable. Ridge Regression provides an alternative to this hard selection of variables that splits them into incldued in and excluded from the model. Instead, it penalizes the coefficients to shrink them towards zero. Not exactly zero, as that would mean exlusion from the model, but in the direction of zero, which can be viewed as decreasing model’s complexity in a continuous way, while keeping all variables in the model.

In Ridge Regression, the Linear Regression loss function is augmented in such a way to not only minimize the sum of squared residuals but also to penalize the size of parameter estimates:

In [14]:
Image(url= "https://miro.medium.com/max/974/1*wJE2pjOmTATRF78cHabpwQ.png")

Solving this minimization problem results in an analytical formula for the βs:

In [ ]:
Image(url= "https://miro.medium.com/max/264/1*dLXDbWeBRkCpccHD-LrRUA.png")

where I denotes an identity matrix. The penalty term λ is a hyperparameter to be chosen: the larger its value, the more are the coefficients shrinked towards zero. One can see from the formula above that as λ goes to zero, the additive penalty vanishes and β-ridge becomes the same as β-OLS from linear regression. On the other hand, as λ grows to infinity, β-ridge approaches zero: with high enough penalty, coefficients can be shrinked arbitrarily close to zero.

But does this shrinkage really result in reducing the variance of the model at the cost of introducing some bias as promised? Yes, it does, which is clear from the formulae for ridge regression estimates’ bias and variance: as λ increases, so does the bias, while the variance goes down!

In [15]:
Image(url="https://miro.medium.com/max/810/1*_1qpxhlhtLcpfO7dQ_bS9g.png")

Now, how to choose to best value for λ? Run cross-validation trying a set of different values and pick one that minimizes cross-validated error on test data. Luckily, Python’s scikit-learn can do this for us.

In [16]:
ridge_cv = RidgeCV(normalize=True, alphas=np.logspace(-10, 1, 400))
ridge_model = ridge_cv.fit(X_train, y_train)
ridge_prediction = ridge_model.predict(X_test)
ridge_mae = np.mean(np.abs(y_test - ridge_prediction))
ridge_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((ridge_model.intercept_, ridge_model.coef_), 
                                axis=None), 3))
)

print('Ridge Regression MAE: {}'.format(np.round(ridge_mae, 3)))
print('Ridge Regression coefficients:')
ridge_coefs

Ridge Regression MAE: 0.239
Ridge Regression coefficients:


{'Intercept': 0.948,
 'Unnamed: 0': 0.033,
 'age': -0.007,
 'gleason': -0.035,
 'lbph': 0.049,
 'lcavol': 0.154,
 'lcp': -0.051,
 'lweight': 0.076,
 'pgg45': 0.003,
 'svi': 0.088}

# LASSO -> L1 Regularization

Lasso, or Least Absolute Shrinkage and Selection Operator, is very similar in spirit to Ridge Regression. It also adds a penalty for non-zero coefficients to the loss function, but unlike Ridge Regression which penalizes sum of squared coefficients (the so-called L2 penalty), LASSO penalizes the sum of their absolute values (L1 penalty). As a result, for high values of λ, many coefficients are exactly zeroed under LASSO, which is never the case in Ridge Regression.

Another important difference between them is how they tackle the issue of multicollinearity between the features. In Ridge Regression, the coefficients of correlated variables tend be similar, while in LASSO one of them is usually zeroed and the other is assigned the entire impact. Because of this, Ridge Regression is expected to work better if there are many large parameters of about the same value, i.e. when most predictors truly impact the response. LASSO, on the other hand, is expected to come on top when there are a small number of significant parameters and the others are close to zero, i.e. when only a few predictors actually influence the response.

In practice, however, one doesn’t know the true values of the parameters. So, the choice between Ridge Regression and LASSO can be based on out-of-sample prediction error. Another option is to combine these two approaches in one — see the next section!

LASSO’s loss function looks as follows:

In [18]:
Image(url="https://miro.medium.com/max/620/1*X1KWVThORJD3H72jctDmhQ.png")

Unlike in Ridge Regression, this minimization problem cannot be solved analytically. Fortunately, there are numerical algorithms able to deal with it.

In [19]:
lasso_cv = LassoCV(normalize=True, alphas=np.logspace(-10, 1, 400))
lasso_model = lasso_cv.fit(X_train, y_train)
lasso_prediction = lasso_model.predict(X_test)
lasso_mae = np.mean(np.abs(y_test - lasso_prediction))
lasso_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((lasso_model.intercept_, lasso_model.coef_), axis=None), 3))
)

print('LASSO MAE: {}'.format(np.round(lasso_mae, 3)))
print('LASSO coefficients:')
lasso_coefs

LASSO MAE: 0.222
LASSO coefficients:


{'Intercept': 0.597,
 'Unnamed: 0': 0.035,
 'age': -0.0,
 'gleason': 0.0,
 'lbph': 0.021,
 'lcavol': 0.096,
 'lcp': 0.0,
 'lweight': 0.0,
 'pgg45': 0.0,
 'svi': 0.0}

Elastic Net

Elastic Net first emerged as a result of critique on LASSO, whose variable selection can be too dependent on data and thus unstable. Its solution is to combine the penalties of Ridge Regression and LASSO to get the best of both worlds. Elastic Net aims at minimizing the loss function that includes both the L1 and L2 penalties:

In [20]:
Image(url="https://miro.medium.com/max/942/1*435ISABMIkEjwvkBDrpsQQ.png")

where α is the mixing paramter between Ridge Regression (when it is zero) and LASSO (when it is one). The best α can be chosen with scikit-learn’s cross-validation-based hyperparaneter tuning.

In [21]:
elastic_net_cv = ElasticNetCV(normalize=True, alphas=np.logspace(-10, 1, 400), 
                              l1_ratio=np.linspace(0, 1, 100))
elastic_net_model = elastic_net_cv.fit(X_train, y_train)
elastic_net_prediction = elastic_net_model.predict(X_test)
elastic_net_mae = np.mean(np.abs(y_test - elastic_net_prediction))
elastic_net_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((elastic_net_model.intercept_, 
                                 elastic_net_model.coef_), axis=None), 3))
)

print('Elastic Net MAE: {}'.format(np.round(elastic_net_mae, 3)))
print('Elastic Net coefficients:')
elastic_net_coefs

Elastic Net MAE: 0.222
Elastic Net coefficients:


{'Intercept': 0.597,
 'Unnamed: 0': 0.035,
 'age': -0.0,
 'gleason': 0.0,
 'lbph': 0.021,
 'lcavol': 0.096,
 'lcp': 0.0,
 'lweight': 0.0,
 'pgg45': 0.0,
 'svi': 0.0}

# Least Angle Regression

So far we have discussed one subsetting method, Best Subset Regression, and three shrinkage methods: Ridge Regression, LASSO and their combination, Elastic Net. This section is devoted to an approach located somewhere in between subsetting and shrinking: Least Angle Regression (LAR). This algorithm starts with a null model, with all coefficients equal to zero, and then works iteratively, at each step moving the coefficient of one of the variables towards its least squares value.

More specifically, LAR starts with identifying the variable most correlated with the response. Then it moves the coefficient of this variable continuously toward its leasts squares value, thus decreasing its correlation with the evolving residual. As soon as another variable “catches up” in terms of correlation with the residual, the process is paused. The second variable then joins the active set, i.e. the set of variables with non-zero coefficients, and their coefficients are moved together in a way that keeps their correlations tied and decreasing. This process is continued until all the variables are in the model, and ends at the full least-squares fit. The name “Least Angle Regression” comes from the geometrical interpretation of the algorithm in which the new fit direction at a given step makes the smallest angle with each of the features that already have non-zero coefficents.

The code chunk below applies LAR to the prostate data.

In [22]:
LAR_cv = LarsCV(normalize=True)
LAR_model = LAR_cv.fit(X_train, y_train)
LAR_prediction = LAR_model.predict(X_test)
LAR_mae = np.mean(np.abs(y_test - LAR_prediction))
LAR_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1], 
        np.round(np.concatenate((LAR_model.intercept_, LAR_model.coef_), axis=None), 3))
)

print('Least Angle Regression MAE: {}'.format(np.round(LAR_mae, 3)))
print('Least Angle Regression coefficients:')
LAR_coefs

Least Angle Regression MAE: 0.222
Least Angle Regression coefficients:


{'Intercept': 0.598,
 'Unnamed: 0': 0.035,
 'age': 0.0,
 'gleason': 0.0,
 'lbph': 0.02,
 'lcavol': 0.096,
 'lcp': 0.0,
 'lweight': 0.0,
 'pgg45': 0.0,
 'svi': 0.0}

# Principal Components Regression

We have already discussed methods for choosing variables (subsetting) and decreasing their coefficients (shrinkage). The last two methods explained in this article take a slightly different approach: they squeeze the input space of the original features into a lower-dimensional space. Mainly, they use X to create a small set of new features Z that are linear combinations of X and then use those in regression models.

The first of these two methods is Principal Components Regression. It applies Principal Components Analysis, a method allowing to obtain a set of new features, uncorrelated with each other and having high variance (so that they can explain the variance of the target), and then uses them as features in simple linear regression. This makes it similar to Ridge Regression, as both of them operate on the principal components space of the original features (for PCA-based derivation of Ridge Regression see [1] in Sources at the bottom of this article). The difference is that PCR discards the components with least informative power, while Ridge Regression simply shrinks them stronger.

The number of components to reatain can be viewed as a hyperparameter and tuned via cross-validation, as is the case in the code chunk below.

In [23]:
regression_model = LinearRegression(normalize=True)
pca_model = PCA()
pipe = Pipeline(steps=[('pca', pca_model), ('least_squares', regression_model)])
param_grid = {'pca__n_components': range(1, 9)}
search = GridSearchCV(pipe, param_grid)
pcareg_model = search.fit(X_train, y_train)
pcareg_prediction = pcareg_model.predict(X_test)
pcareg_mae = np.mean(np.abs(y_test - pcareg_prediction))
n_comp = list(pcareg_model.best_params_.values())[0]
pcareg_coefs = dict(
   zip(['Intercept'] + ['PCA_comp_' + str(x) for x in range(1, n_comp + 1)], 
       np.round(np.concatenate((pcareg_model.best_estimator_.steps[1][1].intercept_, 
                                pcareg_model.best_estimator_.steps[1][1].coef_), axis=None), 3))
)

print('Principal Components Regression MAE: {}'.format(np.round(pcareg_mae, 3)))
print('Principal Components Regression coefficients:')
pcareg_coefs

Principal Components Regression MAE: 0.245
Principal Components Regression coefficients:


{'Intercept': 2.452, 'PCA_comp_1': 0.029, 'PCA_comp_2': -0.026}

# Partial Least Squares

The final method discussed in this article is Partial Least Squares (PLS). Similarly to Principal Components Regression, it also uses a small set of linear combinations of the original features. The difference lies in how these combinations are constructed. While Principal Components Regression uses only X themselves to create the derived features Z, Partial Least Squares additionally uses the target y. Hence, while constructing Z, PLS seeks directions that have high variance (as these can explain variance in the target) and high correlation with the target. This stays in contrast to the principal components appraoch, which focuses on high variance only.

Under the hood of the algorithm, the first of the new features, z1, is created as a linear combination of all features X, where each of the Xs is weighted by its inner product with the target y. Then, y is regressed on z1 giving PLS β-coefficients. Finally, all X are orthogonalized with respect to z1. Then the process starts anew for z2 and goes on until the desired numbers of components in Z is obtained. This number, as usual, can be chosen via cross-validation.

It can be shown that although PLS shrinks the low-variance components in Z as desired, it can sometimes inflate the high-variance ones, which might lead to higher prediction errors in some cases. This seems to be the case for our prostate data: PLS performs the worst among all discussed methods.

In [25]:
pls_model_setup = PLSRegression(scale=True)
param_grid = {'n_components': range(1, 9)}
search = GridSearchCV(pls_model_setup, param_grid)
pls_model = search.fit(X_train, y_train)
pls_prediction = pls_model.predict(X_test)
pls_mae = np.mean(np.abs(y_test - pls_prediction))
pls_coefs = dict(
  zip(data.columns.tolist()[:-1], 
      np.round(np.concatenate((pls_model.best_estimator_.coef_), axis=None), 3))
)

print('Partial Least Squares Regression MAE: {}'.format(np.round(pls_mae, 3)))
print('Partial Least Squares Regression coefficients:')
pls_coefs

Partial Least Squares Regression MAE: 1.083
Partial Least Squares Regression coefficients:


{'Unnamed: 0': 0.988,
 'age': -0.045,
 'gleason': -0.028,
 'lbph': 0.056,
 'lcavol': 0.214,
 'lcp': -0.116,
 'lweight': 0.033,
 'pgg45': 0.094,
 'svi': 0.049}

# Recap & Conclusions

With many, possibly correlated features, linear models fail in terms of prediction accuracy and model’s interpretability due to large variance of model’s parameters. This can be alleviated by reducing the variance, which can only happen at the cost of introducing some bias. Yet, finding the best bias-variance trade-off can optimize model’s performance.

Two broad classes of approaches allowing to achieve this are subsetting and shrinkage. The former selects a subset of variables, while the latter shrinks the coefficients of the model towards zero. Both approaches result in a reduction of model’s complexity, which leads to the desired decrease in parameters’ variance.

This article discussed a couple of subsetting and shrinkage methods:

+ **Best Subset Regression** iterates over all possible feature combination to select the best one;
+ **Ridge Regression penalizes** the squared coefficient values (L2 penalty) enforcing them to be small;
+ **LASSO penalizes** the absolute values of the coefficients (L1 penalty) which can force some of them to be exactly zero;
+ **Elastic Net** combines the L1 and L2 penalties, enjoying the best of Ridge and Lasso;
+ **Least Angle Regression** fits in between subsetting and shrinkage: it works iteratively, adding “some part” of one of the features at each step;
+ **Principal Components Regression** performs PCA to squeeze the original features into a small subset of new features and then uses those as predictors;
+ **Partial Least Squares** also summarizes original features into a smaller subset of new ones, but unlike PCR, it also makes use of the targets to construct them.

As you will see from the applications to the prostate data if you run the code chunks above, most of these methods perform similarly in terms of prediction accuracy. The first 5 methods’ errors range between 0.467 and 0.517, beating least squares’ error of 0.523. The last two, PCR and PLS, perform worse, possibily due to the fact that there are not that many features in the data, hence gains from dimensionality reduction are limited.